In [1]:
spark

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 47, 6, Finished, Available, Finished)

In [8]:
workspace_default_storage_account = "projectgstoragedfb938a3e"
workspace_default_container = "azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f"
workspace_wasbs_base_url = f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 47, 22, Finished, Available, Finished)

In [20]:
from pyspark.sql.functions import lit
import pandas

cancer_path = f"{workspace_wasbs_base_url}cancer_subreddit_sentiment.parquet"
cancer_df = spark.read.parquet(cancer_path)

not_cancer_path = f"{workspace_wasbs_base_url}not_cancer_subreddit_sentiment.parquet"
not_cancer = spark.read.parquet(not_cancer_path)

cancer_df = cancer_df.withColumn("source", lit("cancer"))

not_cancer = not_cancer.withColumn("source", lit("non_cancer"))

df = cancer_df.union(not_cancer)
df = df.select('text', 'source')

df = df.toPandas()

df.head()

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 47, 34, Finished, Available, Finished)

,text,source
0,"Check out Northside Hospital -Atlanta, Piedmon...",cancer
1,I had something similar and managed to get it ...,cancer
2,"That's an insulting offer, and I'm a LPN. Out ...",cancer
3,"Yeah sorry, it was my mistake. I meant to say ...",cancer
4,I see my colorectal dr tomorrow. My disease is...,cancer


In [21]:
df = df[['text']]
df = df.rename(columns={'text': 'text_column'})
df.head()

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 47, 35, Finished, Available, Finished)

,text_column
0,"Check out Northside Hospital -Atlanta, Piedmon..."
1,I had something similar and managed to get it ...
2,"That's an insulting offer, and I'm a LPN. Out ..."
3,"Yeah sorry, it was my mistake. I meant to say ..."
4,I see my colorectal dr tomorrow. My disease is...


In [2]:
%pip install nrclex

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 47, 11, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 KB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43311 sha256=f80b0ee2347f052e40d5aa44283af8fd9d769e77ef51b76ef1c8f06503ee582a
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/d2/10/44/6abfb1234298806a145fd6bcaec8cbc712e88dd1cd6cb242fa
Successfully built nrclex
You should consider upgrading via the '/nfs4/pyenv-402ac994-ebe3-47c4-9480-be3453578112/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import nrclex
import nltk
from multiprocessing import Pool

# Download NLTK data for tokenization
nltk.download('punkt')

# Sample dataframe with a text column
#data = {'text_column': ["I love coding!", "I hate bugs in my code.", "This is a neutral statement.", 
#                        "I enjoy solving problems!", "I get frustrated when things don't work."]}
#df = pd.DataFrame(data)

# Define a function to get sentiment from NRC lexicon
def get_sentiment(text):
    if isinstance(text, str):
        # Initialize NRC lexicon for each text
        nrc_lex = nrclex.NRCLex(text)
        affect_frequencies = nrc_lex.affect_frequencies
        
        # Extracting positive, negative, neutral, and other emotions including 'trust'
        sentiment = {
            "positive": affect_frequencies.get("positive", 0),
            "negative": affect_frequencies.get("negative", 0),
            "neutral": affect_frequencies.get("neutral", 0),
            "anger": affect_frequencies.get("anger", 0),
            "fear": affect_frequencies.get("fear", 0),
            "disgust": affect_frequencies.get("disgust", 0),
            "sadness": affect_frequencies.get("sadness", 0),
            "joy": affect_frequencies.get("joy", 0),
            "surprise": affect_frequencies.get("surprise", 0),
            "trust": affect_frequencies.get("trust", 0)  # Added trust emotion
        }
        return sentiment
    return {
        "positive": 0, "negative": 0, "neutral": 0, "anger": 0, "fear": 0, 
        "disgust": 0, "sadness": 0, "joy": 0, "surprise": 0, "trust": 0  # Default 0 for all emotions
    }

# Function to process a chunk of data
def process_chunk(chunk):
    chunk[['positive', 'negative', 'neutral', 'anger', 'fear', 'disgust', 'sadness', 'joy', 'surprise', 'trust']] = \
        chunk['text_column'].apply(get_sentiment).apply(pd.Series)
    return chunk

# Function to split dataframe into chunks
def split_dataframe(df, num_chunks):
    chunk_size = len(df) // num_chunks
    return [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Number of chunks for parallel processing (adjust based on your system's cores)
num_chunks = 4  # or a value that suits your setup

# Step 1: Split the DataFrame into chunks
chunks = split_dataframe(df, num_chunks)

# Step 2: Use multiprocessing to process the chunks in parallel
with Pool(processes=num_chunks) as pool:
    result_chunks = pool.map(process_chunk, chunks)

# Step 3: Combine the processed chunks back into a single DataFrame
final_df = pd.concat(result_chunks, ignore_index=True)

# Display the final dataframe with sentiment columns
print(final_df)


StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 47, 36, Finished, Available, Finished)

[nltk_data] Downloading package punkt to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                             text_column  positive  negative  \
0      Check out Northside Hospital -Atlanta, Piedmon...  0.198020  0.089109   
1      I had something similar and managed to get it ...  0.666667  0.000000   
2      That's an insulting offer, and I'm a LPN. Out ...  0.275862  0.103448   
3      Yeah sorry, it was my mistake. I meant to say ...  0.000000  0.500000   
4      I see my colorectal dr tomorrow. My disease is...  0.000000  0.166667   
...                                                  ...       ...       ...   
19995  Switch update was just a normal firmware updat...  0.000000  0.000000   
19996                                          [removed]  0.000000  0.000000   
19997  When will someone just fucking follow glorious...  0.000000  0.166667   
19998                                  Ew, STFU already.  0.000000  0.000000   
19999  Szemely szerint csak bemutatkoznek neki udvari...  0.000000  0.000000   

       neutral     anger      fear   di